In [2]:
import pandas as pd
import json
import urllib
import re
import datetime
import pytz
import matplotlib.pyplot as plt

filepath='insertYourFilePath'

with open (filepath+"/llWifidata/raw/2018-07-17_05 2.txt", "r",  errors='replace') as myfile:
    data=myfile.readlines()
    dataRawJson=[json.loads(data[i]) for i in range(len(data))]

with open (filepath+"/llWifidata/clean/pri-2018-07-17_05", "r") as myfile:
    data=myfile.readlines()
dataFormatted = '[' + re.sub('}\s*{',  '},{', data[0]) + ']'#search for }{, ignoring whitespace, add a comma 
dataCleanJson=json.loads(dataFormatted)
dataCleanJsonLists=[[rec['data'][i].split(',') for i in range(len(rec['data']))] for rec in dataCleanJson]
validMacs=[[record[i][5] for i in range(len(record))] for record in dataCleanJsonLists]
uniqueValidMacs=set([mac for macList in validMacs for mac in macList])
numUniqueValidMacs=len(uniqueValidMacs)

timeCorrection=8*60*60
allTimes=[]
individuals={}
c=0
for record in dataRawJson:
    c+=1
    print(c)
    wifi=record['wifidata']
    apmac=wifi['apmac']
    ts=float(wifi['tssend'])+timeCorrection
    allTimes.extend([ts])
    dateTime=datetime.datetime.utcfromtimestamp(ts/1000).strftime('%Y%m%d %H:%M:%S')
    count=wifi['count']
    for obs in wifi['wifitalist']:
        valid=obs['mac'] in uniqueValidMacs
        newObs={'probe':apmac,
                'ts':ts,
                'dateTime':dateTime,
                'rssi':obs['rssi']}
        if obs['mac'] in individuals:
            individuals[obs['mac']]['observations'].append(newObs)
        else:
            individuals[obs['mac']]={'valid':valid, 'observations':[newObs]}
            
obsPerPerson=[len(individuals[i]) for i in individuals]
#plt.hist(obsPerPerson, bins=100)

minTime=datetime.datetime.utcfromtimestamp(min(allTimes)/1000).strftime('%Y%m%d %H:%M:%S')
maxTime=datetime.datetime.utcfromtimestamp(max(allTimes)/1000).strftime('%Y%m%d %H:%M:%S')

numValidMacsRaw=sum([individuals[ind]['valid'] for ind in individuals])
numValidObs=sum([individuals[ind]['valid']*len(individuals[ind]['observations'])for ind in individuals])
numObs=sum([len(individuals[ind]['observations']) for ind in individuals])
pcentValid=100*numValidObs/numObs

FileNotFoundError: [Errno 2] No such file or directory: './llWifidata/raw/2018-07-17_05 2.txt'